In [1]:
!pip install opencv-contrib-python

In [2]:
!pip install --upgrade opencv-contrib-python

In [ ]:
!pip install --upgrade opencv-contrib-python # Upgrade opencv to the latest version to ensure all functions are available
import cv2
import numpy as np
import tensorflow as tf

# Load the object detection model from TensorFlow Model Zoo
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
model_dir = tf.keras.utils.get_file(fname='ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
                                    origin=model_url,
                                    untar=True)
model_path = model_dir + "/saved_model"
model = tf.saved_model.load(model_path)

# Initialize the video stream
cap = cv2.VideoCapture('video_stream.mp4')  # Replace with your video source

# Initialize a list for object trackers
trackers = cv2.MultiTracker_create()

def preprocess_frame(frame):
    # Resize and normalize the frame as required by your model
    input_tensor = tf.convert_to_tensor([cv2.resize(frame, (320, 320))], dtype=tf.uint8)
    return input_tensor

def postprocess_detections(detections, frame_shape):
    # Extract bounding box coordinates and confidence scores from detections
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()

    # Filter boxes by a confidence threshold
    threshold = 0.5
    filtered_boxes = boxes[scores > threshold]
    return filtered_boxes

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection on the current frame
    input_tensor = preprocess_frame(frame)
    detections = model(input_tensor)

    # Process the detection results
    detection_boxes = postprocess_detections(detections, frame.shape)

    # Initialize the object trackers with the detected regions
    for box in detection_boxes:
        y1, x1, y2, x2 = box
        x, y, w, h = int(x1 * frame.shape[1]), int(y1 * frame.shape[0]), int((x2 - x1) * frame.shape[1]), int((y2 - y1) * frame.shape[0])
        tracker = cv2.TrackerCSRT_create()
        trackers.add(tracker, frame, (x, y, w, h))

    # Update the object trackers
    success, boxes = trackers.update(frame)

    # Draw bounding boxes and labels on the frame for tracked objects
    for i, new_box in enumerate(boxes):
        x, y, w, h = map(int, new_box)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, f'Object {i+1}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Display the resulting frame with object detection and tracking
    cv2.imshow('Object Detection and Tracking', frame)

    if cv2.waitKey(1) == 27:  # Press 'Esc' to exit
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()